# Basic exercise 5

In [ ]:
import numpy as np
from scipy.integrate import solve_ivp
import matplotlib.pyplot as plt

This is a basic exercise in the course Python For Scientists. 
The aim is to get you acquainted with the syntax of `scipy` and `numpy` and give you the necessary skills to tackle more serious problems later on.

Of course these problems can be solved very easily by using AI tools. However, since the goal is to teach you the basics, it is not recommended to use AI. Try to solve them independetly instead.

## Logistic population growth

In 1838, the Belgian mathematician Pierre François Verhulst (who was a doctor in number theory from our very own Ghent University) published an equation that he called the [logistic equation](https://en.wikipedia.org/wiki/Logistic_function):
\begin{equation}
\frac{dN}{dt} = rN(t)(K-N(t))
\end{equation}
This equation can be used to describe the growth of a colony of bacteria or mice for example and is of great importance in the study of population growth in biology. 
Later, a discrete version of this equation was defined, the [logistic map](https://en.wikipedia.org/wiki/Logistic_map), which is very important in chaos theory and the study of non-linear dyamics. 

In the logistic equation, $N$ represents the number of individuals in a population, $r$ is the intrinsic growth rate and $K$ is the maximal number of individuals the population can reach, for example due to the limiting food supply.
The analytical solution to the logistic equation is the logistic function:
\begin{equation}
N(t) = N(0)\frac{K}{N(0)+\exp{(-rKt)}(K-N(0))}
\end{equation}

### Part 1
Implement the differential equation and it's analytical solution. 
Use `scipy.integrate.solve_ivp` to integrate the system with `N0 = 100`, `r = 1e-2`, `K = 1000` in the time interval $[0, 1]$.
Plot the resulting curve together with the exact solution. Use the Runge–Kutta–Fehlberg-4(5) integrator as method. Plot your integrated curve together with the analytical solution

In [ ]:
# Implement your solution here

In [ ]:
# Logistic growth ODE
def f(t, N, r, K):
    return r * N * (K - N)


def exactsol(N0, r, K, t):
    return (N0 * K) / (N0 + np.exp(-r * K * t) * (K - N0))


# Parameters
N0 = 100
r = 1e-2
K = 1000
t_span = (0, 1.0)  # start and end time
t_eval = np.linspace(t_span[0], t_span[1], 100)  # evaluation points

# Solve using RK45
sol = solve_ivp(
    fun=lambda t, N: f(t, N, r, K), t_span=t_span, y0=[N0], method="RK45", t_eval=t_eval
)

In [ ]:
# Plot
plt.close()
plt.plot(t_eval, exactsol(N0, r, K, t_eval), color="r", label="Exact solution")
plt.plot(sol.t, sol.y[0], "g.", label="RK23 approximation")
plt.xlabel("t")
plt.ylabel("N(t)")
plt.legend()
plt.show()

### Part 2
Calculate the difference between the analytical result and the numerical integration for the last timepoint. Does this error decrease when you increase the number of `t_eval` points? Why/Why not?

In [ ]:
# Implement your solution here

In [ ]:
t_span = (0, 1.0)  # start and end time
nums = [10, 100, 1000, 10000]
diffs = []
for num in nums:
    t_eval = np.linspace(t_span[0], t_span[1], num)  # evaluation points
    sol_num = solve_ivp(
        fun=lambda t, N: f(t, N, r, K),
        t_span=t_span,
        y0=[N0],
        method="RK45",
        t_eval=t_eval,
    )
    sol_analytical = exactsol(N0, r, K, t_eval)
    diff = sol_num.y[0] - sol_analytical
    diffs.append(np.abs(diff[-1]))

plt.close()
plt.scatter(nums, diffs)
plt.xlabel("Number of evaluation points")
plt.ylabel("Error")
plt.xscale("log")
plt.yscale("log")
plt.show()

"""The error does not decrease because the t_eval points are just points where 
the solution is returned (based on interpolation).
The RK45 method uses an adaptive timestep which is chosen 
based on the difference between an RK4 method and an RK5 method. The solution is 
calculated using these timepoints, but only returned for the t_eval points."""

### Part 3

Fix the step size $h$ of the integration by chosing `max_step` = $h$ and `min_step` = $h$. Vary $h$ from $10^{-1}$ to $10^{-5}$ and calculate the error of the last timepoint of the numerical solution compared to the analytical solution. What is the order of the `RK45` integration method?

In [ ]:
# Implement your solution here

In [ ]:
t_span = (0, 1.0)  # start and end time
stepsizes = [1e-1, 1e-2, 1e-3, 1e-4, 1e-5]
diffs = []
for h in stepsizes:
    t_eval = np.linspace(t_span[0], t_span[1], 1000)  # evaluation points
    sol_num = solve_ivp(
        fun=lambda t, N: f(t, N, r, K),
        t_span=t_span,
        y0=[N0],
        method="RK45",
        t_eval=t_eval,
        max_step=h,
        min_step=h,
    )
    sol_analytical = exactsol(N0, r, K, t_eval)
    diff = sol_num.y[0] - sol_analytical
    diffs.append(np.abs(diff[-1]))

plt.close()
plt.scatter(stepsizes, diffs)
plt.xlabel("stepsize")
plt.ylabel("Error")
plt.xscale("log")
plt.yscale("log")
plt.show()

"""The method of 4th order: a decrease in stepsize with a factor 10 leads to
a decrease in the error of a factor 10^4. This goes on until you hit the 
machine accuracy (about 10^-12), after which rounding errors start to dominate
and the error goes up again."""